# Figure 6 - Acceleration quantification

Create the figure panels describing the algorithmic acceleration provided by the SONIC model validation against the detailed NICE model.

### Imports

In [12]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
from PySONIC.core import NeuronalBilayerSonophore, PulsedProtocol, AcousticDrive
from PySONIC.utils import logger, si_format
from PySONIC.neurons import getPointNeuron
from PySONIC.plt import cm2inch
from utils import saveFigsAsPDF, subdirectory, subthr, suprathr

logger.setLevel(logging.INFO)

### Functions

In [3]:
def getLookupsCompTime(pneuron):
    ''' Get the total computation time for specific lookups. '''
    nbls = NeuronalBilayerSonophore(32e-9, pneuron)
    tcomps4D = nbls.getLookup(keep_tcomp=True).squeeze()['tcomp']
    return np.sum(tcomps4D)

def plotComparativeCompTimes(x, xlabel, comptimes_dict, colors, time_indicators, tcomp_lookup=None,
                             xscale='lin', figsize=cm2inch(11, 11.6), fs=15, lw=2, ps=8, legend=False):
    # Backbone
    fig, ax = plt.subplots(figsize=figsize)
    plt.subplots_adjust(bottom=0.2, left=0.25, right=0.95, top=0.95)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlabel(xlabel, fontsize=fs, labelpad=1)
    ax.set_ylabel('Computation time (s)', fontsize=fs)
    if xscale == 'log':
        ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylim((1e-1, 1e6))
    
    # Time indicators
    axis_to_data = ax.transAxes + ax.transData.inverted()
    data_to_axis = axis_to_data.inverted()
    for t, lbl in time_indicators:
        ax.axhline(t, linewidth=0.5, linestyle='--', c='k')
        yt = data_to_axis.transform((0, tt)[1])
        ax.text(
            1.3, yt, lbl, transform=ax.transAxes,
            horizontalalignment='left', verticalalignment='center', fontsize=fs)
    
    # Hide minor y ticks
    ax.get_yaxis().set_tick_params(which='minor', size=0)
    ax.get_yaxis().set_tick_params(which='minor', width=0)
    
    # Add lookups comp time if provided
    if tcomp_lookup is not None:
        ax.axhline(tcomp_lookup, color='k', linewidth=lw)
    
    # Plot comparative comp times
    for i, key in enumerate(comptimes_dict):
        ax.plot(x, comptimes_dict[key], 'o--', color=colors[i], linewidth=lw, label=key, markersize=ps)
    
    # Set labels fontsize
    for item in ax.get_yticklabels():
        item.set_fontsize(fs)
    for item in ax.get_xticklabels():
        item.set_fontsize(fs)
    
    # Add legend if specified
    if legend:
        ax.legend(frameon=False, fontsize=fs)
    
    return fig

### Data sub-directory

In [4]:
subdir = subdirectory('comparisons')

### Plot parameters

In [5]:
figindex = 6
time_indicators = zip(
    [1, 60, 60**2, 60**2 * 24, 60**2 * 24 * 7],
    ['1 s', '1 min', '1 hour', '1 day', '1 week'])
figs = {}

### Simulation parameters

In [6]:
a = 32e-9  # m
Adrive = 100e3  # Pa
Fdrive = 500e3  # Hz
tstim = 150e-3  # s
toffset = 100e-3  # s
PRF = 100.  # Hz
DC = 1.0
cov = 1.0

CW_pp = PulsedProtocol(tstim, toffset)
methods = ['full', 'sonic']

In [7]:
pneuron = getPointNeuron('RS')
nbls = NeuronalBilayerSonophore(a, pneuron)
tcomp_lookup = getLookupsCompTime(pneuron)

## Panel A: acceleration across US amplitudes (CW, RS neuron)

Comparison of computation times vs. amplitude for CW stimuli

In [8]:
# Amplitudes
Athr = nbls.titrate(AcousticDrive(Fdrive), CW_pp)
amps = np.sort(np.hstack((
    np.array([subthr(Athr), Athr, suprathr(Athr)]),
    np.array([50, 100, 300, 600]) * 1e3)))  # Pa

# Get computation times
comptimes_vs_amps = {
    key: np.array([
        nbls.getOutput(AcousticDrive(Fdrive, x), CW_pp, cov, method, None, outputdir=subdir)[1]['tcomp']
        for x in amps
    ])
    for method in methods}

# Plot
figs['a'] = plotComparativeCompTimes(
    amps * 1e-3, 'Amplitude (kPa)',
    comptimes_vs_amps, ['silver', 'dimgrey'],
    time_indicators, tcomp_lookup=tcomp_lookup, xscale='log')

 28/04/2020 17:27:06: NeuronalBilayerSonophore(32.0 nm, CorticalRS): full simulation @ f = 500kHz, A = 36.09kPa, tstim = 150ms, toffset = 100ms


KeyboardInterrupt: 

## Panel B: acceleration across US frequencies (CW, RS neuron)

Comparison of computation times vs. US frequency for CW stimuli

In [9]:
# Frequencies
freqs = np.array([20e3, 100e3, 500e3, 1e6, 2e6, 3e6, 4e6])  # Hz

# Get computation times
comptimes_vs_freqs = {k: [] for k in methods}
for x in freqs:
    A = suprathr(nbls.titrate(AcousticDrive(x), CW_pp))  # Pa
    for method in methods:
        comptimes_vs_freqs[method].append(
            nbls.getOutput(AcousticDrive(x, A), CW_pp, cov, method, None, outputdir=subdir)[1]['tcomp'])

# Plot
figs['b'] = plotComparativeCompTimes(
    freqs * 1e-3, 'Frequency (kHz)',
    comptimes_vs_freqs, ['silver', 'dimgrey'],
    time_indicators, tcomp_lookup=tcomp_lookup, xscale='log')

 28/04/2020 17:27:12: NeuronalBilayerSonophore(32.0 nm, CorticalRS): full simulation @ f = 20kHz, A = 35.43kPa, tstim = 150ms, toffset = 100ms


KeyboardInterrupt: 

## Panel C: acceleration across sonophore radii (CW, RS neuron)

Comparison of computation times vs. sonophore radius for CW stimuli

In [10]:
# Sonophore radii
radii = np.array([16, 22.6, 32, 45.3, 64]) * 1e-9  # nm

# Get computation times
comptimes_vs_radii = {k: [] for k in methods}
for x in radii:
    nbls = NeuronalBilayerSonophore(x, pneuron)
    A = suprathr(nbls.titrate(AcousticDrive(Fdrive), CW_pp))  # Pa
    for method in methods:
        comptimes_vs_radii[method].append(
            nbls.getOutput(AcousticDrive(Fdrive, A), CW_pp, cov, method, None, outputdir=subdir)[1]['tcomp'])

# Plot
figs['c'] = plotComparativeCompTimes(
    radii * 1e9, 'Sonophore radius (nm)',
    comptimes_vs_radii, ['silver', 'dimgrey'],
    time_indicators, tcomp_lookup=tcomp_lookup, xscale='log')

 28/04/2020 17:27:17: NeuronalBilayerSonophore(16.0 nm, CorticalRS): full simulation @ f = 500kHz, A = 76.02kPa, tstim = 150ms, toffset = 100ms


KeyboardInterrupt: 

## Panel D: acceleration across duty cycles (RS and LTS neuron)

Comparison of computation times vs. duty cycle for RS and LTS neurons

In [11]:
# Duty cycles and neurons
DCs = np.array([5, 10, 25, 50, 75, 100]) * 1e-2
neurons = ['RS', 'LTS']

# Get computation times
comptimes_vs_DC = {}
key = lambda neuron, method: f'{neuron}_{method}'
for neuron in neurons:
    for method in methods:
        comptimes_vs_DC[key(neuron, method)] = []
    pneuron = getPointNeuron(neuron)
    nbls = NeuronalBilayerSonophore(a, pneuron)
    for x in DCs:
        for method in methods:
            comptimes_vs_DC[key(neuron, method)].append(
                nbls.getOutput(
                    AcousticDrive(Fdrive, Adrive),
                    PulsedProtocol(tstim, toffset, PRF, x),
                    cov, method, None, outputdir=subdir
                )[1]['tcomp'])

# Plot
colors = list(plt.get_cmap('Paired').colors[:6])
del colors[2:4]
figs['d'] = plotComparativeCompTimes(
    DCs * 1e2, 'Duty cycle (%)',
    comptimes_vs_DC, colors, time_indicators, legend=True)

 28/04/2020 17:27:24: NeuronalBilayerSonophore(32.0 nm, CorticalRS): full simulation @ f = 500kHz, A = 100.00kPa, tstim = 150ms, toffset = 100ms, PRF = 100.00Hz, DC = 5.0%


KeyboardInterrupt: 

### Save figure panels

Save figure panels as **pdf** in the *figs* sub-folder:

In [20]:
saveFigsAsPDF(figs, figindex)